# Sales Prediction for Time Series Data

This notebook will generate features.  Since the data given is in the daily method, while the predictions are for monthly level, the first step is to aggregate daily data into monthly data.
<br>
The features generated include: 
1. date related features to capture seasonality.
2. revenues with different groups of aggregation 
        shop_id + date_block_num
        shop_id + date_block_num + category_id
3. item count with different groups of aggregation
        date_block_num
        shop_id + date_block_num
        shop_id + date_block_num + category_id
4. cumulative item count for each item
5. flag for new items
6. price with different groups of aggregation
        weighted mean price by item_id + date_block_num
        average price by item_id + date_block_num
        average price by item_id + date_block_num + store_id
7. for each date_block_num, the rank of category revenue by shop_id
8. for each date_block_num, the rank of shop_id by number of items sold
9. for each date_block_num,, the rank of shop_id by revenue
10. for each date_block_num, the price of an item for a store relative to (weighted, mean, median, max, min) price of all the stores.
11. for each date_block_num, the revenue of an item_category for a shop relative to total revenue of the shop
12. The percentage of the specific item sold by a shop_id in a month relative to 
        total items sold by the shop in a month 
        total items sold by the shop in a month for the 'category' of the specific item 
        total items sold for the specific 'item' in a month by ALL the shops
        total items sold for the specific 'item' by ALL the shops cumulatively
13. Lags of numeric features for the past 1-3 months, and 12-months.
14. Difference between lag-1 and lag-2 numeric features.
15. One hot encode 'month', 'year', 'item_category_id', 'shop_id'

## Part 2: Feature Engineering


### Import cleaned data

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import os
#import math
from sklearn.preprocessing import OneHotEncoder, Imputer
%matplotlib inline 

In [2]:
# %load helperFunctions.py
def show_info(dfList):
    '''print out information for a list of dataframes
    Argument: list of dataframe names
    Return: none
    '''
    for df in dfList:
        name = df.name
        print(name)
        print(df.head())
        
        print("------Description of Dataset -------" )
        print(df.describe())

        print("------Description of Dtypes --------" )
        print(df.dtypes)

        print("------Grouping by Dtypes --------" )
        print(df.columns.to_series().groupby(df.dtypes).groups)               # group by data types

        print("------count of unique values -------" )
        print(df.nunique())

        print("------count of np.nan -------" )
        print(df.isnull().sum())

        print("------zero variance numeric columns -------" )
        print(df.std()[df.std() == 0].index.values)
    
    return

def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# column comparison
def col_comparison(df1, df2):
    '''
    compare two dataframes to see which columns are in both set, 
    in set1 only, and in set2 only
    arguments: df1, df2
    returns: samecols, df1only, df2only
    '''
    df1Cols = set(list(df1.columns.values))
    df2Cols = set(list(df2.columns.values))
    samecols = list(df1Cols.intersection(df2Cols))
    df1only = list(df1Cols & set(df1Cols ^ df2Cols))
    df2only = list(df2Cols & set(df1Cols ^ df2Cols))    
    df1name =[x for x in globals() if globals()[x] is df1][0]
    df2name =[x for x in globals() if globals()[x] is df2][0]    
    print('------ columns in both sets -----')
    print(samecols)
    print('')
    print('------ columns in %s only ----- ' % (df1name))
    print(df1only)
    print('')
    print('------ columns in %s only ----- ' % (df2name))
    print(df2only)
    
    return samecols, df1only, df2only
  

# define columns to add drop so train and test set matches
def make_cols_match(train, test, y_name):
    '''define columns to add drop so two datasets matches
    Arguments: train, test, y_name (dataframe 1, dataframe 2, output variable y in quotes)
    Returns: train, test
    '''
    print("---columns comparsion prior to change---")
    samecols, df1only, df2only = col_comparison(train, test)
    train_y = train[y_name]

    # remove columns only in train
    train = remove_columns(train, df1only)
    
    # remove columns only in test
    test = remove_columns(test, df2only)
    
    # append y_name to test and train
    train[y_name] = train_y
    test[y_name] = '0'

    
    return train, test

# show near-zero variance numeric columns
def nearZeroCols(df, threshold):
    '''show near-zero variance numeric columns <= to a threshold
    Arguments: df, threshold
    Returns: array of column names<= threshold
    '''
    return df.std()[df.std() <= threshold].index.values

# change data type
def cast_dtype(df, cols, dtype):
    ''' 
    change datatype of dataframe given column names
    Arguments: df: dataframe, cols: list of column names [], dtype: data type to cast to i.e.: 'int64', 'float64', 'str'
    Returns: dataframe
    '''
    df[cols] = df[cols].astype(dtype)
    
    return df  


def cast_datetime(df, cols):
    '''
    change datatype of dataframe to datetime given column names
    Arguments: df: dataframe, cols: column names in [' ', ' ']
    Returns: dataframe
    '''
    df.loc[:, cols] = df.loc[:, cols].apply(pd.to_datetime, errors='coerce')
    
    return df
 

# text processing
def process_text(text):
    '''
    perform a variety of transformations to text
    Argument: text
    Returns: text
    '''
    text = text.replace(r'.0', '')                  # remove .0 from the end of strings
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    text = text.lstrip('0')                         # strip leading 0
    text = ' '.join(text.split())                   # strip multiple whitespaces into 1
    text = text.strip()                             # strip whitespace
    text = text.lower()                             # convert to lower case
    
    return text

def textProcessing(df, cols):
    '''
    apply transformation of text to specified columns in a dataframe using function 'process_text'
    Arguments: dataframe, list of column names to transform
    Returns: dataframe'''
    
    if cols is None: 
        stringCols = df.loc[:, df.dtypes == object].columns
        dateTimeCols = df.select_dtypes(include=['datetime64']).columns
        cols = list(set(stringCols)-set(dateTimeCols))
    
    df.loc[:, cols] = df.loc[:, cols].applymap(lambda x: process_text(x))
    
    return df

# remove selected columns
def remove_columns(df, cols):
    ''' remove list of columns given a dataframe.  Error will not rise if column names are not in the dataframe
    Arguments: dataframe, list of column names
    Returns: dataframe'''
    
    cols = list(set(cols).intersection(df.columns))  # to avoid error, delete the named columns in dataframe's columns
    df = df.drop(cols, axis = 1)
    return df

# keep columns that are in both sets only
def keep_columns_in_both_sets(df1, df2):
    ''' given two dataframes, only keep columns that are in both dataframes
    Arguments: dataframe1, dataframe2
    Returns: dataframe1, dataframe2 '''
    
    cols = list(set(list(df1.columns.values)).intersection(list(df2.columns.values)))
    df1 = df1.loc[:,cols]
    df2 = df2.loc[:,cols]
    return df1, df2

# check for common form to signify missing observations, i.e. "0" or "nan" or "blank"
def missing_obs_count_by_column(df, cols, stringlist):
    ''' count the number of observations per columns (only string columns) in a dataframe
    given a list of values to match to 
    
    Arguments: dataframe, columns to search, strings to match
    Returns: print out report'''
    
    if stringlist is None:    
        stringlist = ['0', 'nan', '', 'missing', 'none', 'na']
    
    if cols is None:
        stringCols = df.loc[:, df.dtypes == object].columns
        dateTimeCols = df.select_dtypes(include=['datetime64']).columns
        cols = list(set(stringCols)-set(dateTimeCols))

    for string in stringlist:
        print('-----' + string + '------')
        print(np.sum(df.loc[:, cols].apply(lambda x: x == string)))
    return 


# replace strings
def replaceStrings(df, cols, pattern, replaceTo):
    ''' given a datframe and columns, replace the values of the observations given a pattern to match, and value to replace to
    Arguments: dataframe, columns, pattern to match, value to replace to
    Retruns: dataframe
    '''
    for col in cols:
        #df.loc[col] = df.loc[col].replace(pattern, replaceTo)
        df[col] = df[col].replace(pattern, replaceTo)
    return df


# drop near-zero variance numeric columns
def drop_features_based_on_std_threshold(df, threshold):
    '''given a dataframe, and threshold, drop columns with standard deviations
    that are under or equal to the threshold
    Arguments: dataframe, threshold
    Returns: dataframe '''
    
    uint8Cols = df.loc[:, df.dtypes == 'uint8'].columns
    float64Cols = df.loc[:, df.dtypes == 'float64'].columns
    cols = uint8Cols.append(float64Cols)
    
    stdCalc = df.loc[:,cols].apply(lambda x: np.std(x))
    sparseFeatures = stdCalc[stdCalc <= threshold].index
    
    df = df.drop(sparseFeatures, axis = 1)
    return df
    
    # This implementation is very slow
    #df = df.drop(df.std()[df.std() <= threshold].index.values, axis=1)
    #return df


# show uint8 columns with less than a threshold
def show_features_fewer_than_count(df, threshold):
    '''
    show uint8 columns with less than a threshold
    given a dataframe, filter columns that are uin8 (dummy variables)
    return series showing features with counts fewer than the threshold
    Arguments: dataframe, threshold for filtering count
    Return: a filtered named series with counts of none-zeros
    '''
    
    uint8Cols = df.loc[:, df.dtypes == 'uint8'].columns
    numCount = df.loc[:,uint8Cols].apply(lambda x: np.sum(x))
    sparseFeatures = numCount[numCount <= threshold]
    
    return sparseFeatures

# remove uint8 columns with less than a threshold
def remove_features_fewer_than_count(df, threshold):
    '''
    remove uint8 columns with less than a threshold
    given a dataframe, filter columns that are uin8 (dummy variables)
    delete sparse features from dataframe
    Arguments: dataframe, threshold for filtering count
    Return: dataframe
    '''
    uint8Cols = df.loc[:, df.dtypes == 'uint8'].columns
    numCount = df.loc[:,uint8Cols].apply(lambda x: np.sum(x))
    sparseFeatures = numCount[numCount <= threshold].index
    
    df = df.drop(sparseFeatures, axis = 1)
    return df


Import data

In [3]:
data = pd.read_pickle('../data/data_raw')
data = downcast_dtypes(data)

In [4]:
DATA_FOLDER = '../data'
shops = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
items = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_cats = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
shops.name = 'shops'
items.name = 'items'
item_cats.name = 'item_cats'

### Aggregate daily data and construct monthly sales data frame. 

Create a grid to store data with unique shop_id, item_id combinations for each date_block_num.  Since the sales data only has rows with transactions, creating the grid will fill in 0 for item_shop combinations that are not sold for the month (date_block_num)

In [5]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

In [6]:
from itertools import product #This tool computes the cartesian product of input iterables
grid = [] 
for block_num in data['date_block_num'].unique():
    cur_shops = data.loc[data['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = data.loc[data['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

Turn the grid into a dataframe

In [7]:
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

Calculate the target value: aggregated values for (shop_id, item_id, month)

In [8]:
summary = data.groupby(index_cols, as_index=False).agg({'item_cnt_day': ['sum']})
summary.columns = ['shop_id', 'item_id', 'date_block_num', 'target_item_cnt']

Join aggregated data to the grid

In [9]:
all_data = pd.merge(grid, summary, how='left', on=index_cols).fillna(0)

In [10]:
del grid
gc.collect();

Sort the data

In [11]:
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

Clip data before calculating more features

In [12]:
all_data['target_item_cnt'] = all_data['target_item_cnt'].clip(0, 20, axis=0)

In [13]:
# all_data.to_pickle('data_3')

### Features: Date Related

In [14]:
# all_data = pd.read_pickle('data_3')

In [15]:
summary = data[['date_block_num', 'yymm', 'month', 'year', 'set']].drop_duplicates().reset_index(drop = True)
summary.head()

,date_block_num,yymm,month,year,set
0,0,1301,1,2013,train
1,1,1302,2,2013,train
2,2,1303,3,2013,train
3,3,1304,4,2013,train
4,4,1305,5,2013,train


In [16]:
all_data = pd.merge(all_data, summary, how='left', on= ['date_block_num'])

#### feature: count of each of the weekday for each month

Add the count of each of the weekday for the date_block_num (yymm).  For Nov. 2015, manually add the count. <br> (Monday=0, Sunday=6) <br>
0 Monday 5 <br>
1 Tuesday 4 <br>
2 Wednesday 4 <br>
3 Thursday 4 <br>
4 Friday 4 <br>
5 Saturday 4 <br>
6 Sunday 5



In [17]:
# weekdayTable  = pd.read_pickle('weekdayTable ')

In [18]:
weekdayTable = data[['yymm', 'day', 'weekday']].drop_duplicates()
weekdayTable = weekdayTable.groupby(['yymm', 'weekday']).count()
weekdayTable = weekdayTable.reset_index()
weekdayTable.columns = ['yymm', 'weekday', 'weekdayCount']
weekdayTable = weekdayTable.drop(weekdayTable.index[[238]])
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 0, 'weekdayCount': 5}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 1, 'weekdayCount': 4}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 2, 'weekdayCount': 4}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 3, 'weekdayCount': 4}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 4, 'weekdayCount': 4}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 5, 'weekdayCount': 4}, ignore_index=True)
weekdayTable = weekdayTable.append({'yymm':1511, 'weekday': 6, 'weekdayCount': 5}, ignore_index=True)
weekdayTable.tail(3)

,yymm,weekday,weekdayCount
242,1511,4,4
243,1511,5,4
244,1511,6,5


In [19]:
weekdayTable = weekdayTable.pivot(index='yymm', columns='weekday', values='weekdayCount')
weekdayTable.tail(3)

weekday,0,1,2,3,4,5,6
yymm,,,,,,,
1508,5,4,4,4,4,5,5
1509,4,5,5,4,4,4,4
1510,4,4,4,5,5,5,4


In [20]:
weekdayTable = weekdayTable.reset_index(drop = False)

In [21]:
weekdayTable.columns = ['yymm', 'num_Mon', 'num_Tues', 'num_Wed', 'num_Thu', 'num_Fri', 'num_Sat', 'num_Sun']

In [22]:
col_list = list(weekdayTable.columns.values)[1:]
weekdayTable['daysInMonth'] = weekdayTable[col_list].sum(axis=1)

In [23]:
weekdayTable.head()

,yymm,num_Mon,num_Tues,num_Wed,num_Thu,num_Fri,num_Sat,num_Sun,daysInMonth
0,1511,5,4,4,4,4,4,5,30
1,1301,4,5,5,5,4,4,4,31
2,1302,4,4,4,4,4,4,4,28
3,1303,4,4,4,4,5,5,5,31
4,1304,5,5,4,4,4,4,4,30


In [24]:
weekdayTable['yymm'] = weekdayTable['yymm'].astype('int32')

In [25]:
weekdayTable.sort_values(['yymm'],inplace=True)
weekdayTable = weekdayTable.reset_index(drop = True)
weekdayTable.tail()

,yymm,num_Mon,num_Tues,num_Wed,num_Thu,num_Fri,num_Sat,num_Sun,daysInMonth
30,1507,4,4,5,5,5,4,4,31
31,1508,5,4,4,4,4,5,5,31
32,1509,4,5,5,4,4,4,4,30
33,1510,4,4,4,5,5,5,4,31
34,1511,5,4,4,4,4,4,5,30


In [26]:
weekdayTable['yymm'] = weekdayTable['yymm'].astype('str')

In [27]:
weekdayTable.to_pickle('weekdayTable')

In [28]:
all_data = pd.merge(all_data, weekdayTable, how='left', on= ['yymm'])

In [29]:
all_data = downcast_dtypes(all_data)

In [30]:
#all_data.to_pickle('data_4')

### Features: add features from shops, items, item_cats

In [31]:
# all_data = pd.read_pickle('data_4')

In [32]:
shops = downcast_dtypes(shops)
items = downcast_dtypes(items)
item_cats = downcast_dtypes(item_cats)

In [33]:
all_data = all_data.merge(shops, how = 'left', on = 'shop_id')
all_data = all_data.merge(items, how = 'left', on = 'item_id')
all_data = all_data.merge(item_cats, how = 'left', on = 'item_category_id')

In [34]:
#all_data.to_pickle('data_5')

### Features: Revenue Related

#### feature: aggregate total revenue for the month by ['shop_id', 'date_block_num']

In [35]:
# all_data = pd.read_pickle('data_5')

In [36]:
index_cols = ['shop_id', 'date_block_num']
summary = data.groupby(index_cols, as_index=False).agg({'revenue': ['sum']})
summary.columns = ['shop_id', 'date_block_num', 'shop_revenue']
print('Count of Null Before:', summary.iloc[:,-1].isnull().sum())
summary = summary.fillna(0).reset_index(drop = True)
print('Count of Null After:', summary.iloc[:,-1].isnull().sum())
print(summary.sort_values('shop_revenue')[1:5])
print(summary.tail())

Count of Null Before: 0
Count of Null After: 0
      shop_id  date_block_num  shop_revenue
983        38              34           0.0
1004       39              34           0.0
345        14              34           0.0
1050       41              34           0.0
      shop_id  date_block_num  shop_revenue
1623       59              30      863379.0
1624       59              31      956183.0
1625       59              32     1100853.0
1626       59              33      974885.0
1627       59              34           0.0


Set negative revenue to 0

In [37]:
mask = summary['shop_revenue'] < 0
summary.loc[mask, 'shop_revenue'] = 0

In [38]:
all_data = pd.merge(all_data, summary, how='left', on= ['shop_id', 'date_block_num'])

In [39]:
# all_data[(all_data.shop_revenue == 0) & (all_data.date_block_num != 34)]

#### feature: aggregate total revenue for the month by shop_id + category_id

In [40]:
index_cols = ['shop_id', 'date_block_num', 'item_category_id']
summary = data.groupby(index_cols, as_index=False).agg({'revenue': ['sum']})
summary.columns = ['shop_id', 'date_block_num', 'item_category_id', 'category_revenue']
print('Count of Null Before:', summary.iloc[:,-1].isnull().sum())
summary = summary.fillna(0).reset_index(drop = True)
print('Count of Null After:', summary.iloc[:,-1].isnull().sum())

Count of Null Before: 0
Count of Null After: 0


In [41]:
summary['category_revenue'].describe()

count    6.779300e+04
mean     4.887253e+04
std      1.109358e+05
min      0.000000e+00
25%      3.698000e+03
50%      1.468700e+04
75%      4.576940e+04
max      4.527748e+06
Name: category_revenue, dtype: float64

In [42]:
all_data = pd.merge(all_data, summary, how='left', on= ['shop_id', 'date_block_num', 'item_category_id']).fillna(0)

In [43]:
all_data[(all_data.category_revenue == 0) & (all_data.date_block_num != 34)].shape

(1551853, 22)

In [44]:
#all_data.to_pickle('data_6')

### Features: Item Count Related

In [45]:
#all_data = pd.read_pickle('data_6')

#### feature: aggregate total item_cnt_day for the month by shop_id

In [46]:
index_cols = ['shop_id', 'date_block_num']
summary = data.groupby(index_cols, as_index=False).agg({'item_cnt_day': ['sum']})
summary.columns = ['shop_id', 'date_block_num', 'shop_item_cnt']
print('Count of Null Before:', summary.iloc[:,-1].isnull().sum())
summary = summary.fillna(0).reset_index(drop = True)
print('Count of Null After:', summary.iloc[:,-1].isnull().sum())
print('Sorted Values: \n',summary.sort_values(summary.columns[-1])[1:10])
print('Tail: \n', summary.tail())

Count of Null Before: 0
Count of Null After: 0
Sorted Values: 
       shop_id  date_block_num  shop_item_cnt
1397       52              34            0.0
913        36              34            0.0
911        35              34            0.0
1432       53              34            0.0
142         5              34            0.0
876        34              34            0.0
1526       56              34            0.0
542        21              34            0.0
859        33              27            0.0
Tail: 
       shop_id  date_block_num  shop_item_cnt
1623       59              30          998.0
1624       59              31         1156.0
1625       59              32          915.0
1626       59              33          792.0
1627       59              34            0.0


In [47]:
all_data = pd.merge(all_data, summary, how='left', on= ['shop_id', 'date_block_num'])

#### feature: aggregate total item_cnt_day for the month by shop_id + category_id

In [48]:
index_cols = ['shop_id', 'date_block_num', 'item_category_id']
summary = data.groupby(index_cols, as_index=False).agg({'item_cnt_day': ['sum']})
summary.columns = ['shop_id', 'date_block_num', 'item_category_id', 'category_item_cnt']
print('Count of Null Before:', summary.iloc[:,-1].isnull().sum())
summary = summary.fillna(0).reset_index(drop = True)
print('Count of Null After:', summary.iloc[:,-1].isnull().sum())
print('Sorted Values: \n',summary.sort_values(summary.columns[-1])[1:10])
print('Tail: \n', summary.tail())

Count of Null Before: 0
Count of Null After: 0
Sorted Values: 
        shop_id  date_block_num  item_category_id  category_item_cnt
35876       31              34                33                0.0
35877       31              34                34                0.0
35878       31              34                35                0.0
35879       31              34                36                0.0
35880       31              34                37                0.0
35881       31              34                38                0.0
35882       31              34                40                0.0
35883       31              34                41                0.0
35884       31              34                42                0.0
Tail: 
        shop_id  date_block_num  item_category_id  category_item_cnt
67788       59              34                76                0.0
67789       59              34                77                0.0
67790       59              34              

In [49]:
all_data = pd.merge(all_data, summary, how='left', on= ['shop_id', 'date_block_num', 'item_category_id']).fillna(0)

In [50]:
all_data[(all_data.category_item_cnt == 0) & (all_data.date_block_num != 34)].shape

(1551853, 24)

#### feature: aggregate total item sold groupby ['date_block_num', 'item_id'] and cumulative item sold 

In [51]:
index_cols = ['date_block_num', 'item_id']
grouped = all_data.groupby(index_cols, as_index=False).agg({'target_item_cnt': ['sum']})
grouped.columns = [col[0] if col[-1]=='' else 'month_item_cnt' for col in grouped.columns.values] 
grouped = grouped.sort_values(['item_id', 'date_block_num'])
grouped = grouped.fillna(0).reset_index(drop = True)
grouped['cum_month_item_cnt'] = grouped.groupby('item_id')['month_item_cnt'].cumsum()

In [52]:
grouped.head()

,date_block_num,item_id,month_item_cnt,cum_month_item_cnt
0,20,0,1.0,1.0
1,15,1,2.0,2.0
2,18,1,1.0,3.0
3,19,1,1.0,4.0
4,20,1,1.0,5.0


In [53]:
all_data = pd.merge(all_data, grouped, how='left', on= ['date_block_num', 'item_id'])

#### feature: flag for new item (0 cumulative sale in previous month, but > 1 in current month)

In [54]:
from itertools import product #This tool computes the cartesian product of input iterables
grid = []
block_num = data['date_block_num'].unique()
cur_items = data['item_id'].unique() 
print('# unique items:', data['item_id'].nunique())
print('# block num:',data['date_block_num'].nunique() )

# unique items: 22169
# block num: 35


In [55]:
grid.append(np.array(list(product(*[cur_items, block_num])),dtype='int32'))

In [56]:
grid = pd.DataFrame(np.vstack(grid), columns = ['item_id', 'date_block_num'], dtype=np.int32)
grid = grid.sort_values(['item_id', 'date_block_num'])
grid = grid.reset_index(drop = True)

In [57]:
cumTable = all_data[['item_id', 'date_block_num', 'cum_month_item_cnt']]

In [58]:
cumTable = cumTable.drop_duplicates()

In [59]:
grid = pd.merge(grid, cumTable, how = 'left', on = ['date_block_num', 'item_id'])

Fill cum_month_item_cnt as 0 if date_block_num = 0

In [60]:
mask = grid['date_block_num'] == 0
grid.loc[mask, 'cum_month_item_cnt'] = 0

In [61]:
grid = grid.fillna(method='ffill')

In [62]:
grid.shape

(775915, 3)

Append lagged value from prior date_block_num

In [63]:
cols_to_rename = ['cum_month_item_cnt']
month_shift = 1
lag_1_grid = grid.copy(deep = True)
lag_1_grid['date_block_num'] = lag_1_grid['date_block_num'] + month_shift

newNames = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x 
lag_1_grid = lag_1_grid.rename(columns = newNames)

Merge dataframes

In [64]:
index_cols = ['item_id', 'date_block_num']
grid = pd.merge(grid, lag_1_grid, on=index_cols, how='left').fillna(0)

Create flag for new items

In [65]:
mask = grid[(grid.cum_month_item_cnt > 0) & (grid.cum_month_item_cnt_lag_1 == 0)].index

In [66]:
grid['flag_newItem'] = 0

In [67]:
grid.loc[mask, 'flag_newItem'] = 1

In [68]:
grid['flag_newItem'].value_counts()

0    754445
1     21470
Name: flag_newItem, dtype: int64

Merge flag_newItem to all_data

In [69]:
grid = grid[['item_id', 'date_block_num', 'flag_newItem']]

In [70]:
all_data = pd.merge(all_data, grid, how='left', on= ['date_block_num', 'item_id'])

In [71]:
all_data.flag_newItem.value_counts()

0    10131966
1      996038
Name: flag_newItem, dtype: int64

Flag for new items in the test set

In [72]:
all_data[(all_data.flag_newItem == 1) & (all_data.set == "test")]

,shop_id,item_id,date_block_num,target_item_cnt,yymm,month,year,set,num_Mon,num_Tues,...,item_name,item_category_id,item_category_name,shop_revenue,category_revenue,shop_item_cnt,category_item_cnt,month_item_cnt,cum_month_item_cnt,flag_newItem


In [73]:
test_items = all_data[(all_data.set == 'test')]['item_id'].unique()
train_items = all_data[(all_data.set == 'train')]['item_id'].unique()
print('test items: ', test_items.size)
print('train items: ', train_items.size)

test items:  5100
train items:  21806


In [74]:
x = np.isin(test_items, train_items)

In [75]:
testNewItems = test_items[x == 0]

In [76]:
mask = all_data.item_id.isin(testNewItems)
all_data.loc[mask, 'flag_newItem'] = 1

In [77]:
all_data['flag_newItem']= all_data['flag_newItem'].astype('uint8')

In [78]:
all_data['flag_newItem'].value_counts()

0    10116720
1     1011284
Name: flag_newItem, dtype: int64

In [79]:
#all_data.to_pickle('data_7')

### Features: Price Related

In [80]:
# all_data = pd.read_pickle('data_7')

#### feature: weighted mean price for the [item_id, date_block_num]

In [81]:
price = data[['date_block_num', 'item_id', 'item_price', 'item_cnt_day']]

In [82]:
price = price.sort_values(['date_block_num', 'item_id', 'item_price']).reset_index(drop = True)

In [83]:
price['total_items_for_price'] = price.groupby(['date_block_num', 'item_id', 'item_price'])['item_cnt_day'].transform('sum')
price['total_items_date'] = price.groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform('sum')

In [84]:
price['weight'] = price['total_items_for_price']/ price['total_items_date']

In [85]:
price['price_weighted'] = price['item_price'] * price['weight']

In [86]:
price.head(4)

,date_block_num,item_id,item_price,item_cnt_day,total_items_for_price,total_items_date,weight,price_weighted
0,0,19,28.0,1.0,1.0,1.0,1.000000,28.0
1,0,27,1890.0,1.0,2.0,7.0,0.285714,540.0
2,0,27,1890.0,1.0,2.0,7.0,0.285714,540.0
3,0,27,2499.0,1.0,5.0,7.0,0.714286,1785.0


In [87]:
grouped = price[['date_block_num', 'item_id', 'price_weighted']].drop_duplicates()
grouped = grouped.groupby(['date_block_num', 'item_id'])['price_weighted'].sum()  # return a series
grouped = pd.DataFrame(grouped).reset_index()

In [88]:
all_data = all_data.merge(grouped, how = 'left', on = ['date_block_num', 'item_id'])

Check for missing weighted price

In [89]:
all_data[['price_weighted']].isnull().sum()

price_weighted    0
dtype: int64

The rows where nothing was sold will have missing weighted_price.  The strategy to fill the NA will be to "forward fill" in case there are any observations where no item was sold for the month, but the item was sold in previous months.

In [90]:
missingIndex = all_data[all_data['price_weighted'].isnull()].index.values
all_data.iloc[missingIndex][:3]

,shop_id,item_id,date_block_num,target_item_cnt,yymm,month,year,set,num_Mon,num_Tues,...,item_category_id,item_category_name,shop_revenue,category_revenue,shop_item_cnt,category_item_cnt,month_item_cnt,cum_month_item_cnt,flag_newItem,price_weighted


Forward Fill

In [91]:
#all_data['price_weighted'] = all_data.groupby(['date_block_num', 'item_id'])['price_weighted'].apply(lambda x: x.fillna(method='ffill'))

It turns out that there aren't any observations that were forward filled.  The price wasn't there because the item was never sold before.  

In [92]:
all_data[['price_weighted']].isnull().sum()

price_weighted    0
dtype: int64

In [93]:
#all_data.to_pickle('data_8')

#### feature: average price for the [item_id, date_block_num]

In [94]:
# all_data = pd.read_pickle('data_8')

In [95]:
price = data[['date_block_num', 'item_id', 'item_price']]

In [96]:
price = price.sort_values(['date_block_num', 'item_id']).drop_duplicates().reset_index(drop = True)

In [97]:
grouped = price.groupby(['date_block_num', 'item_id']).agg({'item_price': ['mean', 'median', min, max, 'std']})
grouped.columns = grouped.columns.droplevel(level=0)
grouped.head(1)

,,mean,median,min,max,std
date_block_num,item_id,,,,,
0,19,28.0,28.0,28.0,28.0,NaN


Rename columns

In [98]:
cols_to_rename = list(grouped.columns)
aggVar = 'price'
rename_func_output = lambda x: '{}_{}'.format(aggVar, x) if x in cols_to_rename else x
grouped = grouped.rename(columns = rename_func_output)
grouped = grouped.reset_index(drop = False)
grouped.head(1)

,date_block_num,item_id,price_mean,price_median,price_min,price_max,price_std
0,0,19,28.0,28.0,28.0,28.0,NaN


In [99]:
all_data = all_data.merge(grouped, how = 'left', on = ['date_block_num', 'item_id'])

In [100]:
#all_data.to_pickle('data_9')

#### feature: average price for the [item_id, date_block_num, store_id]

In [101]:
# all_data = pd.read_pickle('data_9')

In [102]:
price = data[['date_block_num', 'shop_id', 'item_id', 'item_price']]
index_cols = ['date_block_num', 'shop_id', 'item_id']
price = price.sort_values(index_cols).drop_duplicates().reset_index(drop = True)
grouped = price.groupby(index_cols, as_index = False).agg({'item_price': ['mean']})
print(grouped.columns)

MultiIndex(levels=[['item_price', 'item_id', 'shop_id', 'date_block_num'], ['mean', '']],
           labels=[[3, 2, 1, 0], [1, 1, 1, 0]])


Rename columns for MultiIndex levels (multi index)

In [103]:
grouped.columns = [col[0] if col[-1]=='' else 'price_mean_shop' for col in grouped.columns.values] 

In [104]:
grouped.head(3)

,date_block_num,shop_id,item_id,price_mean_shop
0,0,0,32,221.0
1,0,0,33,347.0
2,0,0,35,247.0


In [105]:
all_data = all_data.merge(grouped, how = 'left', on = index_cols)

In [106]:
all_data[['price_mean_shop']].isnull().sum()

price_mean_shop    9518881
dtype: int64

Forward fill the data takes too long. Fill NA instead by mean price of the item regardless of shop_id.    

In [107]:
# all_data['price_mean_shop'] = all_data.groupby(index_cols)['price_mean_shop'].ffill()
# This takes too long

In [108]:
# all_data['price_mean_shop'] = all_data['price_mean_shop'].fillna(all_data.groupby(['shop_id', 'item_id'])['price_mean_shop'].transform('mean'))
# This method suffers from look ahead bias.  

In [109]:
all_data['price_mean_shop'] = all_data['price_mean_shop'].fillna(all_data['price_mean'])

In [110]:
all_data[['price_mean_shop']].isnull().sum()

price_mean_shop    214200
dtype: int64

In [111]:
#all_data.to_pickle('data_10')

### Features: Rank Encodings

#### feature: rank of category_revenue groupby ['date_block_num', 'shop_id']

In [112]:
# all_data = pd.read_pickle('data_10')

Calculate category_revenue rank

In [113]:
index_cols = ['shop_id', 'date_block_num', 'item_category_id','category_revenue']
summary = all_data[index_cols].drop_duplicates()

In [114]:
index_cols = ['shop_id', 'date_block_num']
summary['rank_cat_revenue'] = summary.groupby(index_cols)['category_revenue'].rank(ascending = False)
summary.head(3)

,shop_id,date_block_num,item_category_id,category_revenue,rank_cat_revenue
0,0,0,40,308301.0,4.0
1,0,0,19,448223.0,1.0
2,0,0,30,186263.0,6.0


In [115]:
index_cols = ['shop_id', 'date_block_num', 'item_category_id','category_revenue']
all_data = all_data.merge(summary, how = 'left', on = index_cols)

#### feature: rank of shop_id by the # of items sold for the date_block_num

In [116]:
index_cols = ['shop_id', 'date_block_num', 'shop_item_cnt']
summary = all_data[index_cols].drop_duplicates()
index_cols = ['date_block_num']
summary['rank_shop_item_cnt'] = summary.groupby(index_cols)['shop_item_cnt'].rank(ascending = False)
summary.head(3)

,shop_id,date_block_num,shop_item_cnt,rank_shop_item_cnt
0,0,0,5578.0,5.0
8115,1,0,2947.0,15.0
16230,2,0,1151.0,42.0


In [117]:
index_cols = ['shop_id', 'date_block_num', 'shop_item_cnt']
all_data = all_data.merge(summary, how = 'left', on = index_cols)

#### feature:  rank of shop_id by the shop revenuefor the date_block_num

In [118]:
index_cols = ['shop_id', 'date_block_num', 'shop_revenue']
summary = all_data[index_cols].drop_duplicates()
index_cols = ['date_block_num']
summary['rank_shop_revenue'] = summary.groupby(index_cols)['shop_revenue'].rank(ascending = False)
summary.head(3)

,shop_id,date_block_num,shop_revenue,rank_shop_revenue
0,0,0,2966412.0,8.0
8115,1,0,1527320.0,26.0
16230,2,0,1091137.0,36.0


In [119]:
index_cols = ['shop_id', 'date_block_num', 'shop_revenue']
all_data = all_data.merge(summary, how = 'left', on = index_cols)

In [120]:
#all_data.to_pickle('data_11')

### Features: Interactions

In [121]:
#all_data = pd.read_pickle('data_11')

#### feature: Price - price specific to a store relative to prices in all the stores 

In [122]:
prices = [ 'price_weighted', 'price_mean', 'price_median', 'price_min','price_max']
for p in prices:
    newName = p + '_diff'
    all_data[newName] = all_data['price_mean_shop'] - all_data[p]   

In [123]:
prices = ['price_weighted_diff', 'price_mean_diff','price_median_diff', 'price_min_diff', 'price_max_diff']
for p in prices:
    newName = p + '_std'
    all_data[newName] = all_data[p]/all_data['price_std']

#### feature: Revenue - percent of revenue from the category specific to a shop

In [124]:
all_data['category_revenue_pct'] = all_data['category_revenue'] / all_data['shop_revenue']

#### feature: Item Counts

The percentage of the specific item sold by a shop_id in a month relative to <br>
1. total items sold by the shop in a month <br>
2. total items sold by the shop in a month for the 'category' of the specific item <br>
3. total items sold for the specific 'item' in a month by ALL the shops <br>
4. total items sold for the specific 'item' by ALL the shops cumulatively <br>

In [125]:
item_counts = ['shop_item_cnt','category_item_cnt', 'month_item_cnt', 'cum_month_item_cnt']
for i in item_counts:
    newName = 'item_cnt_pct_' + i
    all_data[newName] = all_data['target_item_cnt']/ all_data[i]

The percentage of items in specific category relative to all items sold by the shop

In [126]:
all_data['category_pct_shop_item_cnt'] = all_data['category_item_cnt']/all_data['shop_item_cnt']

The percentage of the specific item sold by ALL shops relative to the cumulative count of the specific item sold

In [127]:
all_data['month_item_pct_cum_cnt'] = all_data['month_item_cnt']/all_data['cum_month_item_cnt']

In [128]:
#all_data.to_pickle('data_12')

### Features: Lags

In [129]:
# all_data = pd.read_pickle('data_12')

#### feature: obtain lag_1 and lag_2 for numeric fields

In [130]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
categorical_cols = ['yymm', 'month', 'year', 'set', 'num_Mon', 'num_Tues', 'num_Wed', 'num_Thu',\
                    'num_Fri', 'num_Sat', 'num_Sun', 'daysInMonth', 'shop_name',\
                    'item_name', 'item_category_id', 'item_category_name', 'flag_newItem']
numeric_cols = list(all_data.columns.difference(index_cols + categorical_cols)) 

In [131]:
shift_range = [1, 2]

for month_shift in shift_range:
    train_shift = all_data[index_cols + numeric_cols].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    newNames = lambda x: '{}_lag_{}'.format(x, month_shift) if x in numeric_cols else x # Rename columns to target_lag_1  target_item_lag_1 target_shop_lag_1
    train_shift = train_shift.rename(columns = newNames)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)  # index columns:  ['shop_id', 'item_id', 'date_block_num']

In [132]:
del train_shift
gc.collect()

181

#### feature: difference between lag_1 - lag_2

In [133]:
for cols in numeric_cols: 
    newName = cols + '_1m_diff'
    all_data[newName] = all_data[cols + '_lag_1'] - all_data[cols + '_lag_2']

Verify output to check for accuracy

In [134]:
cols = [col for col in all_data.columns if 'target_item_cnt' in col]
cols = cols + list(['date_block_num', 'shop_id', 'item_id'])
all_data[(all_data.shop_id == 4) & (all_data.item_id == 17717)][cols][1:10]

,target_item_cnt,target_item_cnt_lag_1,target_item_cnt_lag_2,target_item_cnt_1m_diff,date_block_num,shop_id,item_id
404409,8.0,20.0,0.0,20.0,1,4,17717
764156,5.0,8.0,20.0,-12.0,2,4,17717
1145566,0.0,5.0,8.0,-3.0,3,4,17717
1520824,0.0,0.0,5.0,-5.0,4,4,17717
1896504,0.0,0.0,0.0,0.0,5,4,17717
2286293,9.0,0.0,0.0,0.0,6,4,17717
2671831,1.0,9.0,0.0,9.0,7,4,17717
3032824,4.0,1.0,9.0,-8.0,8,4,17717
3387573,5.0,4.0,1.0,3.0,9,4,17717


#### feature: more lags for target_item_cnt

In [135]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
numeric_cols =  ['target_item_cnt']

In [136]:
shift_range = [3, 12]

for month_shift in shift_range:
    train_shift = all_data[index_cols + numeric_cols].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    newNames = lambda x: '{}_lag_{}'.format(x, month_shift) if x in numeric_cols else x 
    train_shift = train_shift.rename(columns = newNames)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)  # index columns:  ['shop_id', 'item_id', 'date_block_num']

In [137]:
all_data['target_item_cnt_mean_3m'] = all_data[['target_item_cnt_lag_1', 'target_item_cnt_lag_2','target_item_cnt_lag_3']].mean(axis = 1).fillna(0)

In [138]:
del train_shift
gc.collect()

390

### Downsize data

#### Discard data without lag_12.  Keep only from date_block_num >= 12

In [139]:
all_data = all_data[all_data['date_block_num'] >= 12] 

In [140]:
all_data = downcast_dtypes(all_data)

#### Discard data that would not be available for the target month (i.e. keep lags but discard current month)

In [141]:
index_cols = ['shop_id', 'item_id', 'date_block_num']
categorical_cols = ['yymm', 'month', 'year', 'set', 'num_Mon', 'num_Tues', 'num_Wed', 'num_Thu',\
                    'num_Fri', 'num_Sat', 'num_Sun', 'daysInMonth', 'shop_name',\
                    'item_name', 'item_category_id', 'item_category_name', 'flag_newItem']
numeric_cols = list(all_data.columns.difference(index_cols + categorical_cols)) 
lag_cols = [col for col in all_data[numeric_cols].columns if 'lag' in col]

In [142]:
delete_cols = list(set(numeric_cols) - set(lag_cols))
delete_cols = list(set(delete_cols) - set(['target_item_cnt', 'target_item_cnt_mean_3m']))
keep_cols = sorted(list(set(all_data.columns.difference(delete_cols))))

In [143]:
all_data = all_data[keep_cols]

In [144]:
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
all_data = all_data.reset_index(drop = True)

In [145]:
#all_data.to_pickle('data_13')

### One-hot encoding

In [146]:
# all_data = pd.read_pickle('data_13')

Onehot encoding for dates and category id.  Keep the features also as numeric.  

In [147]:
categorical_cols = ['month', 'year', 'item_category_id', 'shop_id']

In [148]:
onehotData = all_data[categorical_cols].astype(object)

In [149]:
onehotData = pd.get_dummies(data = onehotData)

In [150]:
all_data = pd.concat([all_data, onehotData], axis=1)

In [151]:
all_data.to_pickle('../data/all_data_unscaled_clipped')